## Natural language processing and non-negative matrix factorization (NMF) for topic modeling

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import NMF, LatentDirichletAllocation
import manu_utils as m
m.set_plotting_style_2()

# Magic command to enable plotting inside notebook
%matplotlib inline
# Magic command to enable svg format in plots
%config InlineBackend.figure_format = 'svg'

## Natural language processing (NLP) vocabulary

Text is by itself an unstructured data type. 

* corpus: set of documents
* stop words: 
* tokenization: Tokenization breaks unstructured data, in this case text, into chunks of information which can be counted as discrete elements. These counts of token occurrences in a document can be used directly as a vector representing that document. This immediately turns an unstructured string (text document) into a structured, numerical data structure suitable for machine learning. 
* n-gram : 

In [9]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1,
                             remove=('headers', 'footers', 'quotes'))
documents = dataset.data

In [10]:
type(documents)

list

In [11]:
documents[0]

"Well i'm not sure about the story nad it did seem biased. What\nI disagree with is your statement that the U.S. Media is out to\nruin Israels reputation. That is rediculous. The U.S. media is\nthe most pro-israeli media in the world. Having lived in Europe\nI realize that incidences such as the one described in the\nletter have occured. The U.S. media as a whole seem to try to\nignore them. The U.S. is subsidizing Israels existance and the\nEuropeans are not (at least not to the same degree). So I think\nthat might be a reason they report more clearly on the\natrocities.\n\tWhat is a shame is that in Austria, daily reports of\nthe inhuman acts commited by Israeli soldiers and the blessing\nreceived from the Government makes some of the Holocaust guilt\ngo away. After all, look how the Jews are treating other races\nwhen they got power. It is unfortunate.\n"

We can see that each element of the list is a document containing a news article. We will now proceed to transform our document list to a term frequency-inverse document frequency (TF-IDF) matrix. This matrix has a  (`docs`, `words`) shape, where the words will be our features. The values inside the matrix will be TF-IDF that is defined by the following formula. 

\begin{align}
\text{TF-IDF} = \text{TF} \times \log{ \frac{N}{DF}}
\end{align}


Where: 
$\text{TF} = \text{$word_{j}$ counts in  $document_{i}$}$,  $\text{DF (document frequency) =  counts of the documents that contain $word_{j}$}$ , $\text{N = total number of documents}$. 

Notice that if a word is ubiquitous in all documents (e.g. stop words), the term in the right turns to zero. In this sense TF-IDF makes a robust assesment of the word frequency in a document, eliminating the bias of highly repeated words. This makes sense as ubiquitous words throughout different documents might not contain any information about the documents themselves. 

The sci-kit learn package has a great implementation using the `tfidf_vectorizer`. Some important arguments of this function are `max_df` threshold for the proportion of documents that have highly repeated words, (`min_df`is the low limit cutoff, if `int` it is in counts), `stop_words` let's you pick a language to eliminate stop words from, and `max_features` let's you consider the top words term frequency across the corpus.

Let's compute the TF-IDF vectorizer on our documents. 

In [13]:
no_features = 1000

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=no_features,
                                   stop_words='english')

tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()


In [19]:
type(tfidf)

scipy.sparse.csr.csr_matrix

We can see that our TF-IDF matrix is a scipy sparse matrix and that we cannot readily visualize it's components. 

In [15]:
tfidf_feature_names[-1]

'young'

We can see that our features are words.

In [16]:
df_tfidf = pd.DataFrame(tfidf.toarray(), columns=tfidf_feature_names)

df_tfidf.head()

,00,000,01,02,03,04,0d,0t,10,100,...,written,wrong,wrote,x11,xt,year,years,yes,york,young
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.134497,0.0,0.147027,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0


In [17]:
df_tfidf.shape

(11314, 1000)

Now we are ready to extract the topics from our documents.

## Non-negative matrix factorization (NMF)

NMF can be thought of as a clustering technique, as  it finds a decomposition of samples $\matrix{X}$ into two matrices $\matrix{W}$ and $\matrix{H}$ of non-negative elements, by minimizing the distance d between $\matrix{X}$ and the matrix product $\matrix{W}$$\matrix{H}$ . More specifically each matrix represents the following:

$\matrix{W}$ (clusters) = the topics (clusters) discovered from the documents.
$\matrix{H}$ (coefficient matrix) = the membership weights for the topics in each document.
$\matrix{X}$ (Document-word matrix) — input that contains which words appear in which documents.

\begin{align}
\matrix{X} = \matrix{W}\matrix{H}
\end{align}

We won't go to any mathematical detail but just know that the current implementation in scikit learn uses the minimization of the Frobenius norm, the matrix analog of the euclidean distance, and that you can use different divergence measurements like the Kullback-leibler divergence by modifying the `beta_loss` parameter. 

In [29]:
n_topics = 20

# Run NMF on the TF-IDF matrix
nmf = NMF(n_components=n_topics, max_df=0.95, min_df=2, 
          max_features=no_features, stop_words='english').fit(tfidf)

In [69]:
#Transform
nmf_W = nmf.transform(tfidf)
nmf_H = nmf.components_


In [72]:
nmf_W.shape

(11314, 20)

In [76]:
pd.DataFrame(nmf_W).tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
11309,0.015747,0.0,0.0,0.000000,0.001803,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000
11310,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000
11311,0.000616,0.0,0.0,0.010849,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.040133,0.034209,0.00000,0.000000,0.000000,0.0,0.000000
11312,0.002394,0.0,0.0,0.000000,0.008236,0.001891,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.095761,0.0,0.002302
11313,0.031814,0.0,0.0,0.030299,0.003680,0.000000,0.0,0.0,0.0,0.0,0.027403,0.0,0.0,0.014768,0.016983,0.00344,0.042375,0.015896,0.0,0.000000


We can see that the $\matrix{W}$ matrix has a (`n_documents`, `n_topics`) shape.

In [71]:
nmf_H.shape

(20, 1000)

We can readily see that the NMF $\matrix{H}$ matrix has a size of (`n_topics`, `n_features`)

In [67]:
def display_topics(model, feature_names, no_top_words):
    
    for topic_idx, topic in enumerate(model.components_):
        
        #print topic index
        print ("Topic %d:" % (topic_idx))
        
        #print topic 
        print (" ".join([feature_names[i] for i in topic.argsort()[: - no_top_words -1 :-1]]))
        

In [68]:
no_top_words = 10
display_topics(nmf, tfidf_feature_names, no_top_words)

Topic 0:
people time right did good said say make way government
Topic 1:
window problem using server application screen display motif manager running
Topic 2:
god jesus bible christ faith believe christian christians sin church
Topic 3:
game team year games season players play hockey win league
Topic 4:
new 00 sale 10 price offer shipping condition 20 15
Topic 5:
thanks mail advance hi looking info help information address appreciated
Topic 6:
windows file files dos program version ftp ms directory running
Topic 7:
edu soon cs university ftp internet article email pub david
Topic 8:
key chip clipper encryption keys escrow government public algorithm nsa
Topic 9:
drive scsi drives hard disk ide floppy controller cd mac
Topic 10:
just ll thought tell oh little fine work wanted mean
Topic 11:
does know anybody mean work say doesn help exist program
Topic 12:
card video monitor cards drivers bus vga driver color memory
Topic 13:
like sounds looks look bike sound lot things really thing
To

Voilà. We have our topics and the most important words associated with them. We can also use another method for topic modelling in text called Latent Dirichlet Allocation (LDA). The implementation is very similar, but if you want to see how to implement it follow this [great post](https://medium.com/mlreview/topic-modeling-with-scikit-learn-e80d33668730). Moreover, it is good for you to know that NMF can be applied for collaborative filtering and image analysis. Happy coding!
